# Experience

## Import module

In [1]:
from utils import BasicDataset
from unet import UNet
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.scenarios.dataset_scenario import benchmark_from_datasets
from avalanche.evaluation.metrics import WeightCheckpoint, gpu_usage_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, disk_usage_metrics
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.logging import InteractiveLogger, TextLogger, WandBLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.supervised import Naive,EWC
from avalanche.benchmarks.scenarios.supervised import class_incremental_benchmark
from torch import optim
import numpy as np
import wandb

d:\anaconda\envs\cl_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare Dataset

In [2]:
from avalanche.benchmarks import benchmark_from_datasets
from avalanche.benchmarks.utils.data_attribute import DataAttribute
import torch

In [3]:
aval_datasets = []
for i in range(5):
    train_dataset = BasicDataset(images_dir=f"./data/train/img/{i}",
                                mask_dir=f"./data/train/label/{i}")
    aval_dataset = AvalancheDataset([train_dataset],
                                    data_attributes=[
                                                    DataAttribute([0] * len(train_dataset), "targets_task_labels"),
                                                    ])
    aval_datasets.append(aval_dataset)

100%|██████████| 50/50 [00:04<00:00, 12.25it/s]


In [4]:
bm = benchmark_from_datasets(train=aval_datasets,
                             test=[aval_datasets[0]]*len(aval_datasets))

In [5]:
print(f"original training samples = {len(bm.train_stream[0].dataset)}")

original training samples = 51


## Settings

In [6]:
epochs = 1
batch_size= 8
learning_rate = 0.00001
weight_decay = 0.00000001
momentum = 0.999

## Train using NAIVE

In [8]:
func_name = "NAIVE"

In [9]:
model = UNet(n_channels=3, n_classes=4)

In [10]:

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger(f"./log/tb_data/{func_name}")

wandb_logger = WandBLogger(run_name=func_name,
                           path=f"./log/checkpoint/{func_name}",
                           dir=f"./log/wandb/{func_name}")

# log to text file
text_logger = TextLogger(open(f'./log/{func_name}.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: boyifan1 (kirigiri_suzumiya). Use `wandb login --relogin` to force relogin
wandb: WARNING Path ./log/wandb/NAIVE\wandb\ wasn't writable, using system temp directory.


In [11]:
# CREATE THE STRATEGY INSTANCE (NAIVE)
optimizer = optim.RMSprop(model.parameters(),
                          lr=learning_rate, 
                          weight_decay=weight_decay, 
                          momentum=momentum, 
                          foreach=True)

eval_plugin = EvaluationPlugin(
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(experience=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    gpu_usage_metrics(gpu_id=0, minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger, wandb_logger]
)

In [12]:
from avalanche.training import Naive
cl_strategy = Naive(
    model=model, 
    optimizer=optimizer,
    criterion=CrossEntropyLoss(), 
    train_mb_size=batch_size, 
    train_epochs=epochs,
    device="cuda:0",
    eval_mb_size=batch_size,
    evaluator=eval_plugin)



In [14]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for i, experience in enumerate(bm.train_stream):
# if True:
    # print("Start of experience: ", experience.current_experience)
    # print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(bm.test_stream[i]))

Starting experiment...
-- >> Start of training phase << --
100%|██████████| 7/7 [05:44<00:00, 49.18s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream = 54033.4600
	DiskUsage_MB/train_phase/train_stream = 54033.4600
	Loss_Epoch/train_phase/train_stream = 1.4154
	Loss_MB/train_phase/train_stream = 1.2648
	MaxGPU0Usage_Epoch/train_phase/train_stream = 100.0000
	MaxGPU0Usage_MB/train_phase/train_stream = 100.0000
	RunningTime_Epoch/train_phase/train_stream = 10.9573
	Time_Epoch/train_phase/train_stream = 344.1592
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 from test stream --
100%|██████████| 7/7 [00:57<00:00,  8.20s/it]
> Eval on experience 0 from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Exp000 = 43.8914
	DiskUsage_Exp/eval_phase/test_stream/Exp000 = 54034.0537
	Loss_Exp/eval_phase/test_stream/Exp000 = 1.4168
	MaxGPU0Usage_Experience/eval_phase/test

In [17]:
import pickle
pickle.dump(results,open(f"./log/eval/{func_name}.pkl","wb"))

## Train using EWC

In [8]:
func_name = "EWC"

In [9]:
model = UNet(n_channels=3, n_classes=4)

In [10]:
# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger(f"./log/tb_data/{func_name}")

wandb_logger = WandBLogger(run_name=func_name,
                           path=f".\log\checkpoint\{func_name}",
                           dir=f".\log\wandb\{func_name}")

# log to text file
text_logger = TextLogger(open(f'./log/{func_name}.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: boyifan1 (kirigiri_suzumiya). Use `wandb login --relogin` to force relogin
wandb: WARNING Path .\log\wandb\EWC\wandb\ wasn't writable, using system temp directory.


In [11]:
# CREATE THE STRATEGY INSTANCE (EWC)
optimizer = optim.RMSprop(model.parameters(),
                          lr=learning_rate, 
                          weight_decay=weight_decay, 
                          momentum=momentum, 
                          foreach=True)

eval_plugin = EvaluationPlugin(
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(experience=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    gpu_usage_metrics(gpu_id=0, minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger, wandb_logger]
)

In [12]:
from avalanche.training import EWC
cl_strategy = EWC(
    model=model, 
    optimizer=optimizer,
    criterion=CrossEntropyLoss(), 
    ewc_lambda=0.005,
    train_mb_size=batch_size, 
    train_epochs=epochs,
    device="cuda:0",
    eval_mb_size=batch_size,
    evaluator=eval_plugin)




In [13]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for i, experience in enumerate(bm.train_stream):
# if True:
    # print("Start of experience: ", experience.current_experience)
    # print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(bm.test_stream[i]))

Starting experiment...
-- >> Start of training phase << --
100%|██████████| 7/7 [05:48<00:00, 49.84s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream = 54065.7539
	DiskUsage_MB/train_phase/train_stream = 54065.7539
	Loss_Epoch/train_phase/train_stream = 1.5289
	Loss_MB/train_phase/train_stream = 1.3414
	MaxGPU0Usage_Epoch/train_phase/train_stream = 100.0000
	MaxGPU0Usage_MB/train_phase/train_stream = 100.0000
	RunningTime_Epoch/train_phase/train_stream = 9.6164
	Time_Epoch/train_phase/train_stream = 348.7892
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 from test stream --
100%|██████████| 7/7 [01:19<00:00, 11.37s/it]
> Eval on experience 0 from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Exp000 = 32.6071
	DiskUsage_Exp/eval_phase/test_stream/Exp000 = 54067.7373
	Loss_Exp/eval_phase/test_stream/Exp000 = 1.4442
	MaxGPU0Usage_Experience/eval_phase/test_

In [14]:
import pickle
pickle.dump(results,open(f"./log/eval/{func_name}.pkl","wb"))

## Train using LFL

In [7]:
func_name = "LFL"

In [8]:
model = UNet(n_channels=3, n_classes=4)

In [9]:
# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger(f"./log/tb_data/{func_name}")

wandb_logger = WandBLogger(run_name=func_name,
                           path=f".\log\checkpoint\{func_name}",
                           dir=f".\log\wandb\{func_name}")

# log to text file
text_logger = TextLogger(open(f'./log/{func_name}.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: boyifan1 (kirigiri_suzumiya). Use `wandb login --relogin` to force relogin
wandb: WARNING Path .\log\wandb\LFL\wandb\ wasn't writable, using system temp directory.


In [10]:
# CREATE THE STRATEGY INSTANCE (LFL)
optimizer = optim.RMSprop(model.parameters(),
                          lr=learning_rate, 
                          weight_decay=weight_decay, 
                          momentum=momentum, 
                          foreach=True)

eval_plugin = EvaluationPlugin(
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(experience=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    gpu_usage_metrics(gpu_id=0, minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger, wandb_logger]
)

In [11]:
from utils.util import SGLFL
cl_strategy = SGLFL(
    model=model, 
    optimizer=optimizer,
    criterion=CrossEntropyLoss(), 
    lambda_e=0.005,
    train_mb_size=batch_size, 
    train_epochs=epochs,
    device="cuda:0",
    eval_mb_size=batch_size,
    evaluator=eval_plugin)



In [12]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for i, experience in enumerate(bm.train_stream):
# if True:
    # print("Start of experience: ", experience.current_experience)
    # print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(bm.test_stream[i]))

Starting experiment...
-- >> Start of training phase << --
100%|██████████| 7/7 [05:17<00:00, 45.37s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream = 53902.5674
	DiskUsage_MB/train_phase/train_stream = 53902.5674
	Loss_Epoch/train_phase/train_stream = 1.1221
	Loss_MB/train_phase/train_stream = 0.9843
	MaxGPU0Usage_Epoch/train_phase/train_stream = 100.0000
	MaxGPU0Usage_MB/train_phase/train_stream = 100.0000
	RunningTime_Epoch/train_phase/train_stream = 8.8557
	Time_Epoch/train_phase/train_stream = 317.5070
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 from test stream --
100%|██████████| 7/7 [01:01<00:00,  8.80s/it]
> Eval on experience 0 from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Exp000 = 42.5514
	DiskUsage_Exp/eval_phase/test_stream/Exp000 = 53903.1611
	Loss_Exp/eval_phase/test_stream/Exp000 = 1.2879
	MaxGPU0Usage_Experience/eval_phase/test_

In [13]:
import pickle
pickle.dump(results,open(f"./log/eval/{func_name}.pkl","wb"))

## Train using SI

In [7]:
func_name = "SI"

In [8]:
model = UNet(n_channels=3, n_classes=4)

In [9]:
# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger(f"./log/tb_data/{func_name}")

wandb_logger = WandBLogger(run_name=func_name,
                           path=f".\log\checkpoint\{func_name}",
                           dir=f".\log\wandb\{func_name}")

# log to text file
text_logger = TextLogger(open(f'./log/{func_name}.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: boyifan1 (kirigiri_suzumiya). Use `wandb login --relogin` to force relogin
wandb: WARNING Path .\log\wandb\SI\wandb\ wasn't writable, using system temp directory.


In [10]:
# CREATE THE STRATEGY INSTANCE (SI)
optimizer = optim.RMSprop(model.parameters(),
                          lr=learning_rate, 
                          weight_decay=weight_decay, 
                          momentum=momentum, 
                          foreach=True)

eval_plugin = EvaluationPlugin(
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(experience=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    gpu_usage_metrics(gpu_id=0, minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger, wandb_logger]
)

In [11]:
from avalanche.training import SynapticIntelligence
cl_strategy = SynapticIntelligence(
    model=model, 
    optimizer=optimizer,
    criterion=CrossEntropyLoss(), 
    si_lambda=0.005,
    train_mb_size=batch_size, 
    train_epochs=epochs,
    device="cuda:0",
    eval_mb_size=batch_size,
    evaluator=eval_plugin)

In [12]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for i, experience in enumerate(bm.train_stream):
# if True:
    # print("Start of experience: ", experience.current_experience)
    # print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(bm.test_stream[i]))

Starting experiment...
-- >> Start of training phase << --
100%|██████████| 7/7 [07:18<00:00, 62.70s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream = 53930.7344
	DiskUsage_MB/train_phase/train_stream = 53930.7344
	Loss_Epoch/train_phase/train_stream = 1.1688
	Loss_MB/train_phase/train_stream = 1.0098
	MaxGPU0Usage_Epoch/train_phase/train_stream = 100.0000
	MaxGPU0Usage_MB/train_phase/train_stream = 100.0000
	RunningTime_Epoch/train_phase/train_stream = 10.2833
	Time_Epoch/train_phase/train_stream = 438.7538
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 from test stream --
100%|██████████| 7/7 [01:12<00:00, 10.43s/it]
> Eval on experience 0 from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Exp000 = 43.1977
	DiskUsage_Exp/eval_phase/test_stream/Exp000 = 53931.3281
	Loss_Exp/eval_phase/test_stream/Exp000 = 1.2170
	MaxGPU0Usage_Experience/eval_phase/test

In [13]:
import pickle
pickle.dump(results,open(f"./log/eval/{func_name}.pkl","wb"))

## Train using GEM

In [7]:
func_name = "GEM"

In [8]:
model = UNet(n_channels=3, n_classes=4)

In [9]:
# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger(f"./log/tb_data/{func_name}")

wandb_logger = WandBLogger(run_name=func_name,
                           path=f".\log\checkpoint\{func_name}",
                           dir=f".\log\wandb\{func_name}")

# log to text file
text_logger = TextLogger(open(f'./log/{func_name}.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: boyifan1 (kirigiri_suzumiya). Use `wandb login --relogin` to force relogin
wandb: WARNING Path .\log\wandb\GEM\wandb\ wasn't writable, using system temp directory.


In [10]:
# CREATE THE STRATEGY INSTANCE (GEM)
optimizer = optim.RMSprop(model.parameters(),
                          lr=learning_rate, 
                          weight_decay=weight_decay, 
                          momentum=momentum, 
                          foreach=True)

eval_plugin = EvaluationPlugin(
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(experience=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    gpu_usage_metrics(gpu_id=0, minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger, wandb_logger]
)


In [11]:
from avalanche.training import GEM
cl_strategy = GEM(
    model=model, 
    optimizer=optimizer,
    criterion=CrossEntropyLoss(), 
    patterns_per_exp=10,
    train_mb_size=batch_size, 
    train_epochs=epochs,
    device="cuda:0",
    eval_mb_size=batch_size,
    evaluator=eval_plugin)

In [12]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for i, experience in enumerate(bm.train_stream):
# if True:
    # print("Start of experience: ", experience.current_experience)
    # print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(bm.test_stream[i]))

Starting experiment...
-- >> Start of training phase << --
100%|██████████| 7/7 [05:58<00:00, 51.18s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream = 53956.3965
	DiskUsage_MB/train_phase/train_stream = 53956.3965
	Loss_Epoch/train_phase/train_stream = 1.1734
	Loss_MB/train_phase/train_stream = 1.0165
	MaxGPU0Usage_Epoch/train_phase/train_stream = 100.0000
	MaxGPU0Usage_MB/train_phase/train_stream = 100.0000
	RunningTime_Epoch/train_phase/train_stream = 10.2479
	Time_Epoch/train_phase/train_stream = 358.1660
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 from test stream --
100%|██████████| 7/7 [01:06<00:00,  9.52s/it]
> Eval on experience 0 from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Exp000 = 45.1531
	DiskUsage_Exp/eval_phase/test_stream/Exp000 = 53956.9902
	Loss_Exp/eval_phase/test_stream/Exp000 = 1.3576
	MaxGPU0Usage_Experience/eval_phase/test

In [ ]:
import pickle
pickle.dump(results,open(f"./log/eval/{func_name}.pkl","wb"))